## Model Building

In [ ]:
import sagemaker
import numpy as np
import pandas as pd
import random
import pickle

In [ ]:
# set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

In [ ]:
sagemaker_session = sagemaker.Session()

In [ ]:
s3_bucket = ""
s3_prefix = "deepar_model"

In [ ]:
role = sagemaker.get_execution_role()

In [ ]:
region = sagemaker_session.boto_region_name

In [ ]:
image_name = sagemaker.image_uris.retrieve("forecasting-deepar", region)

### Trip Start

In [ ]:
# 15-min
s3_data_path_start = "s3://{}/{}/data_start_poc".format(s3_bucket, s3_prefix)
s3_output_path_start = "s3://{}/{}/output_start_poc".format(s3_bucket, s3_prefix)

# 30-min
#s3_data_path_start = "s3://{}/{}/data_start_poc30min".format(s3_bucket, s3_prefix)
#s3_output_path_start = "s3://{}/{}/output_start_poc30min".format(s3_bucket, s3_prefix)

# 1-hour
#s3_data_path_start = "s3://{}/{}/data_start_poc1hr".format(s3_bucket, s3_prefix)
#s3_output_path_start = "s3://{}/{}/output_start_poc1hr".format(s3_bucket, s3_prefix)

**Training**

**15-min, student T**: Takes about 1 hour to train (58 mins) and to produce the below metrics on the test set
- RMSE: 1.2466856959864956
- mean_absolute_QuantileLoss: 59924.505764324516
- mean_wQuantileLoss: 1.000542739670148
- wQuantileLoss[0.1]: 0.20091203062302534
- wQuantileLoss[0.2]: 0.40111006030543794
- wQuantileLoss[0.3]: 0.6011151921433796
- wQuantileLoss[0.4]: 0.8009969021696943
- wQuantileLoss[0.5]: 1.0008156214438477
- wQuantileLoss[0.6]: 1.2006446410813532
- wQuantileLoss[0.7]: 1.4003967124746162
- wQuantileLoss[0.8]: 1.599928056774745
- wQuantileLoss[0.9]: 1.7989654400152346

**30-min, student T**: Takes about 45 mins to train and to produce the below metrics on the test set
- RMSE: 2.1787557106279745
- mean_absolute_QuantileLoss: 59913.63178701855
- mean_wQuantileLoss: 1.0003611799074759
- wQuantileLoss[0.1]: 0.2005263899929865
- wQuantileLoss[0.2]: 0.40070455537442595
- wQuantileLoss[0.3]: 0.6007735039049544
- wQuantileLoss[0.4]: 0.8007525877532903
- wQuantileLoss[0.5]: 1.00066450281929
- wQuantileLoss[0.6]: 1.2004877124648718
- wQuantileLoss[0.7]: 1.4002120419622592
- wQuantileLoss[0.8]: 1.5998925308562544
- wQuantileLoss[0.9]: 1.7992367940389504

**1-hour, student T**: Takes about 12 mins to train and to produce the below metrics on the test set
- RMSE: 4.293374719083342
- mean_absolute_QuantileLoss: 59851.39373534173
- mean_wQuantileLoss: 0.9993220085377301
- wQuantileLoss[0.1]: 0.20230291235391099
- wQuantileLoss[0.2]: 0.4025155849388989
- wQuantileLoss[0.3]: 0.6020443379453237
- wQuantileLoss[0.4]: 0.8013119550995238
- wQuantileLoss[0.5]: 1.0006762128143234
- wQuantileLoss[0.6]: 1.1999059092022353
- wQuantileLoss[0.7]: 1.3985901767580375
- wQuantileLoss[0.8]: 1.596075543968918
- wQuantileLoss[0.9]: 1.790475443758399

**15-min, negative binomial**: Takes about 30 mins to train and to produce the below metrics on the test set
- RMSE: 1.1641374227395522
- mean_absolute_QuantileLoss: 56580.33333333333
- mean_wQuantileLoss: 0.9447060264030811
- wQuantileLoss[0.1]: 0.2
- wQuantileLoss[0.2]: 0.4
- wQuantileLoss[0.3]: 0.6
- wQuantileLoss[0.4]: 0.8
- wQuantileLoss[0.5]: 1.0
- wQuantileLoss[0.6]: 1.2
- wQuantileLoss[0.7]: 1.3999966606558467
- wQuantileLoss[0.8]: 1.585200026714753
- wQuantileLoss[0.9]: 1.3171575502571296

In [ ]:
start_estimator = sagemaker.estimator.Estimator(
    image_uri = image_name,
    sagemaker_session = sagemaker_session,
    role = role,
    instance_count = 1,
    instance_type = "ml.c5.2xlarge",
    base_job_name = "deepar-poc-start",
    output_path = s3_output_path_start,
)

In [ ]:
# 15-min
freq = "15min"
context_length = 4 * 24 * 3
prediction_length = 4 * 24 * 3

# 30-min
#freq = "30min"
#context_length = 2 * 24 * 3
#prediction_length = 2 * 24 * 3

# 1-hour
#freq = "H"
#context_length = 24 * 3
#prediction_length = 24 * 3

In [ ]:
start_hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "likelihood": "negative-binomial",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
}

In [ ]:
start_estimator.set_hyperparameters(**start_hyperparameters)

In [ ]:
#%%time
#start_data_channels = {"train": "{}/train_start/".format(s3_data_path_start), "test": "{}/test_start/".format(s3_data_path_start)}

#start_estimator.fit(inputs = start_data_channels, wait = True)

**Prediction**

In [ ]:
import matplotlib.pyplot as plt
from sagemaker.serverless import ServerlessInferenceConfig

from deepar_model_utils import DeepARPredictor
from deepar_model_utils import get_station_data
from deepar_model_utils import prep_station_data

Reference for following code: [stackoverflow](https://stackoverflow.com/questions/56255154/how-to-use-a-pretrained-model-from-s3-to-predict-some-data), [Model docs](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html])

In [ ]:
#start_file = "s3://{}/model_trips_start_station_20208029_20220831.csv".format(s3_bucket)
start_file = "model_trips_start_station_20208029_20220831.csv"
trips_start = pd.read_csv(start_file, parse_dates = True)
trips_start.shape

In [ ]:
trips_start_all_group = prep_station_data(trips_start, "start station id", "starttime")
print(sum(trips_start_all_group["size"]))

In [ ]:
trip_start_model = sagemaker.model.Model(
    # 15-min
    #model_data = "{}/deepar-poc-start-2022-10-19-16-11-42-997/output/model.tar.gz".format(s3_output_path_start),
    # 30-min
    #model_data = "{}/deepar-poc30min-start-2022-11-06-18-03-53-382/output/model.tar.gz".format(s3_output_path_start),
    # 1-hour
    #model_data = "{}/deepar-poc1hr-start-2022-11-06-19-34-46-739/output/model.tar.gz".format(s3_output_path_start),
    # 15-min neg-bin
    model_data = "{}/deepar-poc-start-2022-11-22-19-27-48-129/output/model.tar.gz".format(s3_output_path_start),
    image_uri = image_name,
    role = role,
    predictor_cls = DeepARPredictor, 
    name = "deepar-poc-startmodel")

serverless_config = ServerlessInferenceConfig()

#start_predictor = trip_start_model.deploy(#initial_instance_count = 1, 
#                                          #instance_type = "ml.m5.large", 
#                                          endpoint_name = "deepar-poc-startendpoint", 
#                                          serverless_inference_config = serverless_config)

In [ ]:
#with open('trip_start_model.pkl', 'wb') as f:
#    pickle.dump(trip_start_model, f)

#with open('trip_start_model.pkl', 'rb') as f:
#    trip_start_model_pkl = pickle.load(f)

In [ ]:
print(trip_start_model.name) # model name
print(trip_start_model.endpoint_name) # endpoint name

In [ ]:
freq = "15min" # group and sum trips by a set increment
#freq = "30min" # group and sum trips by a set increment
#freq = "H" # group and sum trips by a set increment

station = 436
train_max_date = "2022-08-28 23:45:00" # make sure all series end at the same time
test_start = "2022-08-29 00:00:00"
max_date = "2022-08-31 23:45:00" # make sure all series end at the same time

In [ ]:
# stations to try: [177, 436, 572, 67]
observed_start = get_station_data(trips_start_all_group, "start station id", "starttime", station, freq, max_date).loc[test_start:]["size"]
predicted_start = start_predictor.predict(ts = get_station_data(trips_start_all_group, "start station id", "starttime", station, freq, train_max_date)["size"], quantiles = [0.025, 0.7, 0.975])

plt.figure(figsize = (12, 6))
observed_start.plot(label = "observed", color = "#808080", alpha = 0.5)
#p2_5 = predicted_start["0.025"]
#p97_5 = predicted_start["0.975"]
#plt.fill_between(p2_5.index, p2_5, p97_5, color = "#808080", alpha = 0.5, label = "95% CI")
predicted_start["0.7"].plot(label = "predicted")

plt.xlabel("date")
plt.ylabel("trip count")
plt.title(str(station))

plt.legend()
plt.show()

**Calculate RMSE**

In [ ]:
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

**15-min, student T**
- Quantile = 0.85; RMSE = 1.02
- **Quantile = 0.90; RMSE = 1.00**
- Quantile = 0.95; RMSE = 1.16

**30-min, student T**
- Quantile = 0.80; RMSE = 1.56
- **Quantile = 0.85; RMSE = 1.49**
- Quantile = 0.90; RMSE = 1.53
- Quantile = 0.95; RMSE = 2.17

**1-hour, student T**
- Quantile = 0.75; RMSE = 2.32
- **Quantile = 0.80; RMSE = 2.29**
- Quantile = 0.85; RMSE = 2.48
- Quantile = 0.90; RMSE = 3.24

**15-min, negative binomial**
- Quantile = 0.65; RMSE = 0.92
- **Quantile = 0.70; RMSE = 0.91**
- Quantile = 0.75; RMSE = 0.93
- Quantile = 0.80; RMSE = 0.98
- Quantile = 0.85; RMSE = 1.07
- Quantile = 0.90; RMSE = 1.25

In [ ]:
quantile = 0.65
y_true = []
y_pred = []

for station in tqdm(trips_start_all_group["start station id"].unique()):
    y_true += get_station_data(trips_start_all_group, "start station id", "starttime", station, freq, max_date, cluster = True, min_date = test_start).loc[test_start:]["size"].tolist()
    y_pred += start_predictor.predict(ts = get_station_data(trips_start_all_group, "start station id", "starttime", station, freq, train_max_date)["size"], quantiles = [quantile])[str(quantile)].tolist()
mean_squared_error(y_true, y_pred, squared = False)

In [ ]:
start_predictor.delete_model()
start_predictor.delete_endpoint()

### Trip Stop

In [ ]:
# 15-min
s3_data_path_stop = "s3://{}/{}/data_stop_poc".format(s3_bucket, s3_prefix)
s3_output_path_stop = "s3://{}/{}/output_stop_poc".format(s3_bucket, s3_prefix)

# 30-min
#s3_data_path_stop = "s3://{}/{}/data_stop_poc30min".format(s3_bucket, s3_prefix)
#s3_output_path_stop = "s3://{}/{}/output_stop_poc30min".format(s3_bucket, s3_prefix)

# 1-hour
#s3_data_path_stop = "s3://{}/{}/data_stop_poc1hr".format(s3_bucket, s3_prefix)
#s3_output_path_stop = "s3://{}/{}/output_stop_poc1hr".format(s3_bucket, s3_prefix)

**Training**

**15-min, student T**: Takes about 1.5 hours to train (90 mins) and to produce the below metrics on the test set
- RMSE: 1.1916626421269132
- mean_absolute_QuantileLoss: 59989.86295699137
- mean_wQuantileLoss: 1.0007984878214171
- wQuantileLoss[0.1]: 0.2010558960899485
- wQuantileLoss[0.2]: 0.40154761764894925
- wQuantileLoss[0.3]: 0.601687960449047
- wQuantileLoss[0.4]: 0.8015763695273128
- wQuantileLoss[0.5]: 1.0013230552798642
- wQuantileLoss[0.6]: 1.2009613247249022
- wQuantileLoss[0.7]: 1.4004675590225628
- wQuantileLoss[0.8]: 1.5998017102259545
- wQuantileLoss[0.9]: 1.7987648974242108

**30-min, student T**: Takes about 30 mins to train and to produce the below metrics on the test set
- RMSE: 2.1231545764037594
- mean_absolute_QuantileLoss: 59935.9481798435
- mean_wQuantileLoss: 0.9998990387348354
- wQuantileLoss[0.1]: 0.20066822408773213
- wQuantileLoss[0.2]: 0.40065327164551484
- wQuantileLoss[0.3]: 0.6004516344835465
- wQuantileLoss[0.4]: 0.8003259611877267
- wQuantileLoss[0.5]: 1.0003178984530885
- wQuantileLoss[0.6]: 1.200186983603307
- wQuantileLoss[0.7]: 1.3998246688000366
- wQuantileLoss[0.8]: 1.5991159027676558
- wQuantileLoss[0.9]: 1.7975468035849091

**1-hour, student T**: Takes about 10 mins to train and to produce the below metrics on the test set
- RMSE: 4.911408997635595
- mean_absolute_QuantileLoss: 59789.733624069406
- mean_wQuantileLoss: 0.9974597715136198
- wQuantileLoss[0.1]: 0.20230946522090493
- wQuantileLoss[0.2]: 0.40355811051773666
- wQuantileLoss[0.3]: 0.6033005662221217
- wQuantileLoss[0.4]: 0.802101273562577
- wQuantileLoss[0.5]: 1.0006246870691806
- wQuantileLoss[0.6]: 1.1985435338384487
- wQuantileLoss[0.7]: 1.3952969431362952
- wQuantileLoss[0.8]: 1.5903024581117946
- wQuantileLoss[0.9]: 1.7811009059435192

**15-min, negative binomial**: Takes about 20 mins to train and to produce the below metrics on the test set
- RMSE: 1.2054183536971828
- mean_absolute_QuantileLoss: 57999.688888888886
- mean_wQuantileLoss: 0.9675968250790579
- wQuantileLoss[0.1]: 0.2
- wQuantileLoss[0.2]: 0.4
- wQuantileLoss[0.3]: 0.6
- wQuantileLoss[0.4]: 0.8
- wQuantileLoss[0.5]: 1.0
- wQuantileLoss[0.6]: 1.2
- wQuantileLoss[0.7]: 1.4
- wQuantileLoss[0.8]: 1.5995529011377667
- wQuantileLoss[0.9]: 1.5088185245737546

In [ ]:
stop_estimator = sagemaker.estimator.Estimator(
    image_uri = image_name,
    sagemaker_session = sagemaker_session,
    role = role,
    instance_count = 1,
    instance_type = "ml.c5.2xlarge",
    base_job_name = "deepar-poc-stop",
    output_path = s3_output_path_stop,
)

In [ ]:
# 15-min
freq = "15min"
context_length = 4 * 24 * 3
prediction_length = 4 * 24 * 3

# 30-min
#freq = "30min"
#context_length = 2 * 24 * 3
#prediction_length = 2 * 24 * 3

# 1-hour
#freq = "H"
#context_length = 24 * 3
#prediction_length = 24 * 3

In [ ]:
stop_hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "likelihood": "negative-binomial",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
}

In [ ]:
stop_estimator.set_hyperparameters(**stop_hyperparameters)

In [ ]:
#%%time
#stop_data_channels = {"train": "{}/train_stop/".format(s3_data_path_stop), "test": "{}/test_stop/".format(s3_data_path_stop)}

#stop_estimator.fit(inputs = stop_data_channels, wait = True)

**Prediction**

In [ ]:
import matplotlib.pyplot as plt
from sagemaker.serverless import ServerlessInferenceConfig

from deepar_model_utils import DeepARPredictor
from deepar_model_utils import get_station_data
from deepar_model_utils import prep_station_data

Reference for following code: [stackoverflow](https://stackoverflow.com/questions/56255154/how-to-use-a-pretrained-model-from-s3-to-predict-some-data), [sagemaker.Model docs](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html])

In [ ]:
#stop_file = "s3://{}/model_trips_stop_station_20208029_20220831.csv".format(s3_bucket)
stop_file = "model_trips_stop_station_20208029_20220831.csv"
trips_stop = pd.read_csv(stop_file, parse_dates = True)
trips_stop.shape

In [ ]:
trips_stop_all_group = prep_station_data(trips_stop, "end station id", "stoptime")
print(sum(trips_stop_all_group["size"]))

In [ ]:
trip_stop_model = sagemaker.model.Model(
    # 15-min
    #model_data = "{}/deepar-poc-stop-2022-10-21-20-01-24-335/output/model.tar.gz".format(s3_output_path_stop),
    # 30-min
    #model_data = "{}/deepar-poc30min-stop-2022-11-06-18-56-59-837/output/model.tar.gz".format(s3_output_path_stop),
    # 1-hour
    #model_data = "{}/deepar-poc1hr-stop-2022-11-06-19-50-56-634/output/model.tar.gz".format(s3_output_path_stop),
    # 15-min neg-bin
    model_data = "{}/deepar-poc-stop-2022-11-22-21-05-30-909/output/model.tar.gz".format(s3_output_path_stop),
    image_uri = image_name,
    role = role,
    predictor_cls = DeepARPredictor, 
    name = "deepar-poc-stopmodel")

serverless_config = ServerlessInferenceConfig()

#stop_predictor = trip_stop_model.deploy(#initial_instance_count = 1, 
#                                        #instance_type = "ml.m5.large", 
#                                        endpoint_name = "deepar-poc-stopendpoint", 
#                                        serverless_inference_config = serverless_config)

In [ ]:
#with open('trip_stop_model.pkl', 'wb') as f:
#    pickle.dump(trip_stop_model, f)

#with open('trip_stop_model.pkl', 'rb') as f:
#    trip_stop_model_pkl = pickle.load(f)

In [ ]:
print(trip_stop_model.name) # model name
print(trip_stop_model.endpoint_name) # endpoint name

In [ ]:
# test: skipping ahead 3 days to forecast
# verdict: able to predict first 3 days in September even though the last 3 days were not used to train model
#station = 68
#freq = "15min"
#max_date = "2022-08-31 23:45:00"
#test_skip = get_station_data(trips_stop_all_group, "end station id", "stoptime", station, freq, max_date)["size"]
#stop_predictor.predict(ts = test_skip, quantiles = [0.7])

In [ ]:
# test: forecast for a station that was not in the training data
# verdict: able to predict the next 3 days for cold start stations
#station = 572
#freq = "15min"
#max_date = "2022-08-29 23:45:00"
#test_cold = get_station_data(trips_stop_all_group, "end station id", "stoptime", station, freq, max_date)["size"]
#stop_predictor.predict(ts = test_cold, quantiles = [0.7])

In [ ]:
freq = "15min" # group and sum trips by a set increment
#freq = "30min" # group and sum trips by a set increment
#freq = "H" # group and sum trips by a set increment

station = 178
train_max_date = "2022-08-28 23:45:00" # make sure all series end at the same time
test_start = "2022-08-29 00:00:00"
max_date = "2022-08-31 23:45:00" # make sure all series end at the same time

In [ ]:
# stations to try: [177, 436, 572, 67]
# note: station 572 results in error because there is no data for that station prior to 8/29/2022
observed_stop = get_station_data(trips_stop_all_group, "end station id", "stoptime", station, freq, max_date).loc[test_start:]["size"]
predicted_stop = stop_predictor.predict(ts = get_station_data(trips_stop_all_group, "end station id", "stoptime", station, freq, train_max_date)["size"], quantiles = [0.025, 0.7, 0.975])

plt.figure(figsize = (12, 6))
observed_stop.plot(label = "observed", color = "#808080", alpha = 0.5)
#p2_5 = predicted_stop["0.025"]
#p97_5 = predicted_stop["0.975"]
#plt.fill_between(p2_5.index, p2_5, p97_5, color = "#808080", alpha = 0.5, label = "95% CI")
predicted_stop["0.7"].plot(label = "predicted")

plt.xlabel("date")
plt.ylabel("trip count")
plt.title(str(station))

plt.legend()
plt.show()

**Calculate RMSE**

In [ ]:
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

**15-min, student T**
- Quantile = 0.75; RMSE = 1.07
- Quantile = 0.85; RMSE = 0.99
- **Quantile = 0.90; RMSE = 0.97**
- Quantile = 0.95; RMSE = 1.11

**30-min, student T**
- Quantile = 0.80; RMSE = 1.50
- **Quantile = 0.85; RMSE = 1.43**
- Quantile = 0.90; RMSE = 1.50
- Quantile = 0.95; RMSE = 2.21

**1-hour, student T**
- Quantile = 0.70; RMSE = 2.23
- **Quantile = 0.75; RMSE = 2.13**
- Quantile = 0.80; RMSE = 2.20
- Quantile = 0.85; RMSE = 2.51
- Quantile = 0.90; RMSE = 3.31

**15-min, negative binomial**
- Quantile = 0.65; RMSE = 0.87
- **Quantile = 0.70; RMSE = 0.87**
- Quantile = 0.75; RMSE = 0.89

In [ ]:
quantile = 0.65
y_true = []
y_pred = []

for station in tqdm(trips_stop_all_group["end station id"].unique()):
    if station != 572:
        y_true += get_station_data(trips_stop_all_group, "end station id", "stoptime", station, freq, max_date, cluster = True, min_date = test_start).loc[test_start:]["size"].tolist()
        y_pred += stop_predictor.predict(ts = get_station_data(trips_stop_all_group, "end station id", "stoptime", station, freq, train_max_date)["size"], quantiles = [quantile])[str(quantile)].tolist()
mean_squared_error(y_true, y_pred, squared = False)

In [ ]:
stop_predictor.delete_model()
stop_predictor.delete_endpoint()